In [1]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_gpu = torch.cuda.is_available()
print(device)

cuda:0


# Initialize for CPU & GPU

In [2]:
if use_gpu:
    !pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric --no-index --find-links=file:///kaggle/input/torch-geometric
#     !pip install torch_geometric_temporal  
else:
    !pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
    !pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
    !pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
    !pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
    !pip install git+https://github.com/pyg-team/pytorch_geometric.git
#     !pip install torch_geometric_temporal


Looking in links: file:///kaggle/input/torch-geometric
Processing /kaggle/input/torch-geometric/torch_scatter-2.1.1-cp310-cp310-linux_x86_64.whl
Processing /kaggle/input/torch-geometric/torch_sparse-0.6.17-cp310-cp310-linux_x86_64.whl
Processing /kaggle/input/torch-geometric/torch_cluster-1.6.1-cp310-cp310-linux_x86_64.whl
Processing /kaggle/input/torch-geometric/torch_spline_conv-1.2.2-cp310-cp310-linux_x86_64.whl
Processing /kaggle/input/torch-geometric/torch_geometric-2.3.1-py3-none-any.whl


# Import

In [3]:
from torch.profiler import profile, record_function, ProfilerActivity
import torch.autograd.profiler as profiler

In [4]:
import numpy as np
import pandas as pd
import os
import re
import json
from scipy.special import perm
from itertools import combinations,chain
from typing import List, Union
from torch_geometric.data import Data
from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt
import pickle
import time
from torch import nn
from torch.nn import Linear as Lin
from torch.nn import ReLU
from torch.nn import Sequential as Seq
import torch.nn.functional as F
from torch_geometric.nn import NNConv, PNAConv
from torch import autograd

from sklearn.model_selection import train_test_split

# DataLoader

In [5]:
Edge_Flag = List[Union[np.ndarray, None]]
Edge_Index = List[Union[np.ndarray, None]]
Edge_Attr = List[Union[np.ndarray, None]]

Node_Flag = List[Union[np.ndarray, None]]
Node_Index = List[Union[np.ndarray, None]]
Node_Attr = List[Union[np.ndarray, None]]
Additional_Attr = List[np.ndarray]



class GraphSignal(object):
    # dynamic node static node attr dynamic edge and edge attr
    def __init__(
        self,
        edge_flag: Edge_Flag,
        edge_index: Edge_Index,
        edge_attr: Edge_Attr,
        node_flag: Node_Flag,
        node_index: Node_Index,
        node_attr: Node_Attr,
        ts_list: List,
    ):
        
        self.edge_flag = torch.LongTensor(edge_flag)
        self.edge_index = torch.LongTensor(edge_index).T.to(device)
        self.edge_attr = edge_attr
        self.node_flag = torch.LongTensor(node_flag)
        self.node_index = torch.LongTensor(node_index).to(device)
        self.node_attr = node_attr
        self.ts_list = ts_list
        self.edge_attr_encoded = None
        self.node_attr_encoded = None
        
        self._set_snapshot_count()

    def _set_snapshot_count(self):
        self.snapshot_count = len(self.edge_flag)
    
    def encode_edge_attr(self, enocder):
        self.edge_attr_encoded = torch.FloatTensor(enocder.transform(self.edge_attr)).to(device)
    
    def encode_node_attr(self, enocder):
        self.node_attr_encoded = torch.FloatTensor(enocder.transform(self.node_attr)).to(device)
        
    def _get_edge_index(self, time_index: int):
        if time_index == 0:
            _start = 0
        else:
            _start = self.edge_flag[time_index-1]
        _end = self.edge_flag[time_index]
        _edge_index = self.edge_index[:,_start:_end]
        return _edge_index

    # def _get_edge_weight(self, time_index: int):
    #     if self.edge_weights[time_index] is None:
    #         return self.edge_weights[time_index]
    #     else:
    #         return torch.FloatTensor(self.edge_weights[time_index])

    def _get_edge_attr(self, time_index: int):
        if self.edge_attr_encoded is None:
            print("Edge Attr Need to be Encoded!")
            raise
        if time_index == 0:
            _start = 0
        else:
            _start = self.edge_flag[time_index-1]
        _end = self.edge_flag[time_index]
        _edge_attr = self.edge_attr_encoded[_start:_end]
        return _edge_attr
    
    def _get_node_index_attr(self, time_index: int):
        if self.node_attr_encoded is None:
            print("Node Attr Need to be Encoded!")
            raise
        if time_index == 0:
            _start = 0
        else:
            _start = self.node_flag[time_index-1]
        _end = self.node_flag[time_index]
        _node_index = self.node_index[_start:_end]
        _node_attr = self.node_attr_encoded[_node_index]
        return _node_index,_node_attr
    
    def _get_timestamp(self, time_index: int):
        _timestamp = self.ts_list[time_index]
        return _timestamp


    def __getitem__(self, time_index: int):
        edge_index = self._get_edge_index(time_index)
        edge_attr = self._get_edge_attr(time_index)
        node_index,node_attr = self._get_node_index_attr(time_index)
        _timestamp = self._get_timestamp(time_index)

        snapshot = Data(
            edge_index=edge_index,
            edge_attr=edge_attr,
            node_index=node_index,
            node_attr=node_attr,
            timestamp = _timestamp
        )
        return snapshot

    def __next__(self):
        if self.t < self.snapshot_count:
            snapshot = self[self.t]
            self.t = self.t + 1
            return snapshot
        else:
            self.t = 0
            raise StopIteration

    def __iter__(self):
        self.t = 0
        return self
    
    def __len__(self):
        return self.snapshot_count


class GraphDatasetLoader(object):
    def __init__(self,input_path=""):
        self.input_path = input_path
        self._read_data()
    
    def _read_data(self):
        self._dataset = np.load(self.input_path)

    def get_dataset(self): # -> DynamicGraphTemporalSignal:
        dataset = GraphSignal(
            edge_flag = self._dataset['edge_flag'],
            edge_index = self._dataset['edge_index'],
            edge_attr = self._dataset['edge_attr'],
            node_flag = self._dataset['node_flag'],
            node_index = self._dataset['node_index'],
            node_attr = self._dataset['node_attr'],
            ts_list = self._dataset['timestamp']
        )
        return dataset

# Model

## MultiPNAConv

In [6]:
class MultiPNAConv(torch.nn.Module):
    def __init__(self, in_channels, out_channels, edge_channels, conv_layers, lin_layers, deg):
        super().__init__()
        
        aggregators = ['mean', 'min', 'max', 'std']
        scalers = ['identity', 'amplification', 'attenuation']
        
        self.convs = nn.ModuleList()
        pre_size = in_channels
        for size in conv_layers:
            conv = PNAConv(in_channels=pre_size, out_channels=size,
                           aggregators=aggregators, scalers=scalers, deg=deg,
                           edge_dim=edge_channels, towers=2, pre_layers=1, post_layers=1,
                           divide_input=False)
            self.convs.append(conv)
            pre_size = size
        
        self.lin_net = Seq()
        for size in lin_layers[:-1]:
            self.lin_net.append(Lin(pre_size,size))
            pre_size = size
        self.lin_net.append(ReLU())
        self.lin_net.append(Lin(pre_size,out_channels))

    def forward(self, x, edge_index, edge_attr):
        out = x
        with profiler.record_function("Graph Conv"):
            for conv in self.convs:
                out = F.relu(conv(out,edge_index,edge_attr))
        out = self.lin_net(out)
        return out

# Graph GRU Unit

In [7]:
class GraphGRU(torch.nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        edge_channels: int,
        conv_layers: List,
        edge_layers: List,
        lin_layers: List,
        deg,
        bias: bool = True,
    ):
        super().__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.edge_channels = edge_channels
        self.conv_layers = conv_layers
        self.edge_layers = edge_layers
        self.lin_layers = lin_layers
        self.deg = deg
        self.bias = bias
        self._create_parameters_and_layers()

    def _create_update_gate_parameters_and_layers(self):

        self.conv_x_z = MultiPNAConv(
            in_channels = self.in_channels,
            out_channels = self.out_channels,
            edge_channels = self.edge_channels,
            conv_layers = self.conv_layers,
            lin_layers = self.lin_layers,
            deg = self.deg,
        )

        self.conv_h_z = MultiPNAConv(
            in_channels = self.out_channels,
            out_channels = self.out_channels,
            edge_channels = self.edge_channels,
            conv_layers = self.conv_layers,
            lin_layers = self.lin_layers,
            deg = self.deg,
        )
        
    def _create_reset_gate_parameters_and_layers(self):

        self.conv_x_r = MultiPNAConv(
            in_channels = self.in_channels,
            out_channels = self.out_channels,
            edge_channels = self.edge_channels,
            conv_layers = self.conv_layers,
            lin_layers = self.lin_layers,
            deg = self.deg,
        )

        self.conv_h_r = MultiPNAConv(
            in_channels = self.out_channels,
            out_channels = self.out_channels,
            edge_channels = self.edge_channels,
            conv_layers = self.conv_layers,
            lin_layers = self.lin_layers,
            deg = self.deg,
        )

    def _create_candidate_state_parameters_and_layers(self):

        self.conv_x_h = MultiPNAConv(
            in_channels = self.in_channels,
            out_channels = self.out_channels,
            edge_channels = self.edge_channels,
            conv_layers = self.conv_layers,
            lin_layers = self.lin_layers,
            deg = self.deg,
        )

        self.conv_h_h = MultiPNAConv(
            in_channels = self.out_channels,
            out_channels = self.out_channels,
            edge_channels = self.edge_channels,
            conv_layers = self.conv_layers,
            lin_layers = self.lin_layers,
            deg = self.deg,
        )

    def _create_parameters_and_layers(self):
        self._create_update_gate_parameters_and_layers()
        self._create_reset_gate_parameters_and_layers()
        self._create_candidate_state_parameters_and_layers()

    def _set_hidden_state(self, X, H):
        if H is None:
            H = torch.zeros(X.shape[0], self.out_channels).to(X.device)
        return H

    def _calculate_update_gate(self, X, edge_index, edge_attr, H):
        Z = self.conv_x_z(X, edge_index, edge_attr)
        Z = Z + self.conv_h_z(H, edge_index, edge_attr)
        Z = torch.sigmoid(Z)
        return Z

    def _calculate_reset_gate(self, X, edge_index, edge_attr, H):
        R = self.conv_x_r(X, edge_index, edge_attr)
        R = R + self.conv_h_r(H, edge_index, edge_attr)
        R = torch.sigmoid(R)
        return R

    def _calculate_candidate_state(self, X, edge_index, edge_attr, H, R):
        H_tilde = self.conv_x_h(X, edge_index, edge_attr)
        H_tilde = H_tilde + self.conv_h_h(H * R, edge_index, edge_attr)
        H_tilde = torch.tanh(H_tilde)
        return H_tilde

    def _calculate_hidden_state(self, Z, H, H_tilde):
        H = Z * H + (1 - Z) * H_tilde
        return H

    def forward(
            self,
            X: torch.FloatTensor,
            edge_index: torch.LongTensor,
            edge_attr: torch.FloatTensor,
            H: torch.FloatTensor = None,
        ) -> torch.FloatTensor:
            H = self._set_hidden_state(X, H)
            Z = self._calculate_update_gate(X, edge_index, edge_attr, H)
            R = self._calculate_reset_gate(X, edge_index, edge_attr, H)
            H_tilde = self._calculate_candidate_state(X, edge_index, edge_attr, H, R)
            H = self._calculate_hidden_state(Z, H, H_tilde)
            return H

# Graph GRU Layer and Global Hidden Function

In [8]:
def create_hidden(num_node, out_channels):
#     hidden_global = torch.FloatTensor(np.zeros([num_node,out_channels])).to(device)
    hidden =torch.zeros([num_node,out_channels], dtype=torch.float).to(device)
    return hidden

def select_hidden(hidden, index):
#     h = hidden_global[index] #REGULAR INDEXING
    h = hidden.index_select(dim=0, index=index) #INDEX SELECT
    return h

# TODO: Aggregation of hidden and cell
def update_hidden(hidden, h, index):
    # hidden_global[index] = h.detach() #REGULAR INDEXING
    # for key,value in mapping.items():
    #     hidden_global[value] = h[key] 
    hidden.index_copy_(dim=0, index=index, source=h.detach())        

class GraphGRULayer(torch.nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        edge_channels: int,
        conv_layers: List,
        edge_layers: List,
        lin_layers: List,
        deg,
        bias: bool = True,
    ):
        super().__init__()
        self.gru = GraphGRU(
            in_channels = in_channels,
            out_channels = out_channels,
            edge_channels = edge_channels,
            conv_layers = conv_layers,
            edge_layers = edge_layers,
            lin_layers = lin_layers,
            deg = deg,
            bias = bias
        )
        
        self.out_channels = out_channels
        
    def forward(
        self,
        X: torch.FloatTensor,
        node_index: torch.LongTensor,
        node_flag: torch.LongTensor,
        edge_index: torch.LongTensor,
        edge_attr: torch.FloatTensor,
        edge_flag: torch.LongTensor,
        num_node: int
    ) -> torch.FloatTensor:
        
        X_split = torch.tensor_split(X, node_flag)
        node_index_split = torch.tensor_split(node_index, node_flag)
        edge_index_split = torch.tensor_split(edge_index, edge_flag, dim=1)
        edge_attr_split = torch.tensor_split(edge_attr, edge_flag)
        
        hidden = create_hidden(num_node, self.out_channels)
        
        outs = []
        for _X,_node_index,_edge_index,_edge_attr in zip(X_split,node_index_split,edge_index_split,edge_attr_split):
            _hidden = select_hidden(hidden, _node_index)
            _new_hidden = self.gru(_X, _edge_index, _edge_attr, _hidden)
            update_hidden(hidden, _new_hidden, _node_index)
            outs.append(_new_hidden)
        
        H = torch.cat(outs)
        return H

## TGAE

In [9]:
class TGAE(torch.nn.Module): # Not Heterogeneous
    def __init__(
        self, in_channels, out_channels, edge_channels, 
        embed_layers, gru_out_channels, decide_layers,
        gru_conv_layers, gru_edge_layers, gru_lin_layers, deg):
        super(TGAE, self).__init__()
        
#         self.in_channels = in_channels
#         self.out_channels = out_channels
#         self.edge_channels = edge_channels
#         self.embed_layers = embed_layers
#         self.gru_out_channels = gru_out_channels
#         self.decide_layers = decide_layers
#         self.gru_conv_layers = gru_conv_layers
#         self.gru_edge_layers = gru_edge_layers
#         self.gru_lin_layers = gru_lin_layers
#         self.deg = deg
        
        # Encoder
        layers = []
        pre_h_num = in_channels
        for h_num in embed_layers[:-1]:
#             layers.append(torch.nn.Dropout(p=0.2))
            layers.append(Lin(pre_h_num,h_num))
            layers.append(torch.nn.LeakyReLU())
            pre_h_num = h_num
        layers.append(Lin(pre_h_num,embed_layers[-1]))
        self.encoder_embedding_net = Seq(*layers)
        
        self.encoder_gru = GraphGRULayer(
            in_channels=embed_layers[-1],
            out_channels=gru_out_channels,
            edge_channels=edge_channels,
            conv_layers=gru_conv_layers,
            edge_layers=gru_edge_layers,
            lin_layers=gru_lin_layers,
            deg=deg
        )

        layers = []
        pre_h_num = gru_out_channels
        for h_num in decide_layers:
#             layers.append(torch.nn.Dropout(p=0.2))
            layers.append(Lin(pre_h_num,h_num))
            layers.append(torch.nn.LeakyReLU())
            pre_h_num = h_num
        layers.append(Lin(pre_h_num,out_channels))
        self.encoder_deciding_net = Seq(*layers)
        
        # Decoder
        layers = []
        pre_h_num = out_channels
        for h_num in decide_layers[::-1]:
            layers.append(torch.nn.Dropout(p=0.2))
            layers.append(Lin(pre_h_num,h_num))
            layers.append(torch.nn.LeakyReLU())
            pre_h_num = h_num
        layers.append(Lin(pre_h_num,gru_out_channels))
        self.decoder_deciding_net = Seq(*layers)
        
        self.decoder_gru = GraphGRULayer(
            in_channels=gru_out_channels,
            out_channels=embed_layers[-1],
            edge_channels=edge_channels,
            conv_layers=gru_conv_layers,
            edge_layers=gru_edge_layers,
            lin_layers=gru_lin_layers,
            deg=deg
        )
        
        layers = []
        pre_h_num = embed_layers[-1]
        for h_num in embed_layers[:-1][::-1]:
            layers.append(torch.nn.Dropout(p=0.2))
            layers.append(Lin(pre_h_num,h_num))
            layers.append(torch.nn.LeakyReLU())
            pre_h_num = h_num
        layers.append(Lin(pre_h_num,in_channels))
        self.decoder_embedding_net = Seq(*layers)
        

    def forward(self, x, node_index, node_flag, edge_index, edge_attr, edge_flag, num_node):
        # Encoder
        out = self.encoder_embedding_net(x)
        
        # GNN layer
        with profiler.record_function("gru encoder"):
            h_encoder = self.encoder_gru(out, node_index, node_flag, edge_index, edge_attr, edge_flag, num_node) 
        
        out = self.encoder_deciding_net(h_encoder)

        out = self.decoder_deciding_net(out)
        
        # TODO Reverse Edge Index 
        with profiler.record_function("gru decoder"):
            h_decoder = self.decoder_gru(out, node_index, node_flag, torch.flip(edge_index,dims=(0,)), edge_attr, edge_flag, num_node)
        
        out = self.decoder_embedding_net(h_decoder)

        return out
    

# Training

## Read Data

In [10]:
data_dir_0 = '/kaggle/input/dissertation-data'
data_dir_1 = '2021-09-11-umbrella-experiment-32run-fran'


signals = []
y = []
with open(os.path.join(data_dir_0, data_dir_1, "annotated.json")) as f:
    annotated_dict = json.load(f)

for data_dir_2 in os.listdir(os.path.join(data_dir_0, data_dir_1)):
    if data_dir_2 == "annotated.json":
        continue
    r = re.compile(".*.npz")
    graph_files = list(filter(r.match, os.listdir(os.path.join(data_dir_0, data_dir_1, data_dir_2))))

    if len(graph_files) > 1:
        print("Multiple Graph Files!")
        raise
    if len(graph_files) == 0:
        print("Not Found Graph File!")
        raise

    dataloader = GraphDatasetLoader(os.path.join(data_dir_0, data_dir_1, data_dir_2, graph_files[0]))
    signal = dataloader.get_dataset()
    signals.append(signal)
    y.append(annotated_dict[data_dir_2])

# split train and test dataset
signals_train, signals_test, y_train, y_test = train_test_split(signals, y, test_size=0.2, random_state=1)
signals_train, signals_val, y_train, y_val = train_test_split(signals_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [11]:
node_attr_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
node_attr_encoder = node_attr_encoder.fit(np.concatenate([sample.node_attr for sample in signals_train]))

edge_attr_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
edge_attr_encoder = edge_attr_encoder.fit(np.concatenate([sample.edge_attr for sample in signals_train]))

for signal in signals_train:
    signal.encode_node_attr(node_attr_encoder)
    signal.encode_edge_attr(edge_attr_encoder)

# X_train = []
# for signal in signals_train:
# #     X_train.append(list(signal))
#     X_train.append(signal.node_attr_encoded.index_select(dim=0,index=signal.node_index))

    
for signal in signals_val:
    signal.encode_node_attr(node_attr_encoder)
    signal.encode_edge_attr(edge_attr_encoder)


# X_val = []
# for signal in signals_val:
# #     X_val.append(list(signal))
#     X_val.append(signal.node_attr_encoded.index_select(dim=0,index=signal.node_index))

num_node_val = []
for signal in signals_val:
    num_node_val.append(signal.node_attr.shape[0])

In [12]:
from torch_geometric.utils import degree
# Compute the maximum in-degree in the training data.
max_degree = -1
    
for signal in signals_train:
    for snapshot in signal:
        d = degree(snapshot.edge_index[1], num_nodes=snapshot.node_attr.shape[0], dtype=torch.long)
        max_degree = max(max_degree, int(d.max()))

# Compute the in-degree histogram tensor
deg = torch.zeros(max_degree + 1, dtype=torch.long).to(device)
for signal in signals_train:
    for snapshot in signal:
        d = degree(snapshot.edge_index[1], num_nodes=snapshot.node_attr.shape[0], dtype=torch.long)
        deg += torch.bincount(d, minlength=deg.numel())

print(deg)

tensor([22546,  6381,  1591,  ...,     0,     0,     1], device='cuda:0')


In [13]:
# for sample in X_train[:1]:
#     for snapshot in sample:
#         node_attr = snapshot.node_attr
#         node_index = snapshot.node_index
#         edge_attr = snapshot.edge_attr
#         edge_index = snapshot.edge_index
        
#         print(f"node_attr: {node_attr.shape}")
#         print(f"node_index: {node_index.shape}")
#         print(f"edge_attr: {edge_attr.shape}")
#         print(f"edge_index: {edge_index.shape}")

In [14]:
IN_CHANNELS = signals_train[0].node_attr_encoded.shape[1]
EDGE_CHANNELS = signals_train[0].edge_attr_encoded.shape[1]

In [15]:
model = TGAE(
    in_channels=IN_CHANNELS, 
    out_channels=5, 
    edge_channels=EDGE_CHANNELS, 
    embed_layers=[16,32],
    gru_out_channels=8,
    decide_layers=[16,16],
    gru_conv_layers=[16,16],
    gru_edge_layers=[32],
    gru_lin_layers=[],
    deg=deg
)

loss_f = torch.nn.MSELoss(reduction = 'none')
optimizer = torch.optim.Adam(model.parameters(), lr = 2e-4, weight_decay=1e-5)

model = model.to(device)
loss_f = loss_f.to(device)
print(model)

TGAE(
  (encoder_embedding_net): Sequential(
    (0): Linear(in_features=5, out_features=16, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=16, out_features=32, bias=True)
  )
  (encoder_gru): GraphGRULayer(
    (gru): GraphGRU(
      (conv_x_z): MultiPNAConv(
        (convs): ModuleList(
          (0): PNAConv(32, 16, towers=2, edge_dim=58)
          (1): PNAConv(16, 16, towers=2, edge_dim=58)
        )
        (lin_net): Sequential(
          (0): ReLU()
          (1): Linear(in_features=16, out_features=8, bias=True)
        )
      )
      (conv_h_z): MultiPNAConv(
        (convs): ModuleList(
          (0): PNAConv(8, 16, towers=2, edge_dim=58)
          (1): PNAConv(16, 16, towers=2, edge_dim=58)
        )
        (lin_net): Sequential(
          (0): ReLU()
          (1): Linear(in_features=16, out_features=8, bias=True)
        )
      )
      (conv_x_r): MultiPNAConv(
        (convs): ModuleList(
          (0): PNAConv(32, 16, towers=2, edge_dim

## Train Loop

In [16]:
# train_loop
def train_loop(signal, model, loss_fn, optimizer, device):
    model.train()
    
    X = signal.node_attr_encoded.index_select(dim=0,index=signal.node_index)
    node_index = signal.node_index
    node_flag = signal.node_flag
    edge_index = signal.edge_index
    edge_attr = signal.edge_attr_encoded
    edge_flag = signal.edge_flag
    
    outs = model(X, node_index, node_flag, edge_index, edge_attr, edge_flag, signal.node_attr.shape[0])

    train_losses = torch.sqrt(torch.sum(loss_f(X, outs),dim=1))
    total_loss = torch.sum(train_losses)
    snapshot_losses = train_losses.detach().index_select(dim=0,index=signal.node_index).cpu().numpy()
    
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    return snapshot_losses

## Test Loop

In [17]:
# test_loop
def test_loop(signal, model, loss_fn, optimizer, device):
    with torch.no_grad():
        X = signal.node_attr_encoded.index_select(dim=0,index=signal.node_index)
        node_index = signal.node_index
        node_flag = signal.node_flag
        edge_index = signal.edge_index
        edge_attr = signal.edge_attr_encoded
        edge_flag = signal.edge_flag

        outs = model(X, node_index, node_flag, edge_index, edge_attr, edge_flag, signal.node_attr.shape[0])

        train_losses = torch.sqrt(torch.sum(loss_f(X, outs),dim=1))
        total_loss = torch.sum(train_losses)
        snapshot_losses = [torch.sum(loss).cpu().numpy() for loss in torch.tensor_split(train_losses.detach(), node_flag)]
    
    return snapshot_losses

In [18]:
GLOBAL_EPOCH = 0
def train_function(num_epoch):
    history_train = []
    history_val = []

    for i_epoch in range(1,num_epoch+1):
        train_losses = []
        _start = time.time()
        for signal in signals_train[:2]:
            snapshot_losses = train_loop(signal, model, loss_f, optimizer, device)
#             print(np.mean(train_loss))
            train_losses.append(np.mean(snapshot_losses))
            
        
        if i_epoch % 5 == 0:
            val_losses = []
            for signal in signals_val:
                snapshot_losses = test_loop(signal, model, loss_f, optimizer, device)
#                 print(np.mean(val_loss))
                val_losses.append(np.mean(snapshot_losses))
            _end = time.time()
            
            history_train.append(np.mean(train_losses))
            history_val.append(np.mean(val_losses))
            print(f"{i_epoch+GLOBAL_EPOCH}/{num_epoch+GLOBAL_EPOCH}: cost {_end-_start:.4f}s train RMSE {np.mean(train_losses):.4f} val RMSE{np.mean(val_losses):.4f}")
        else:
            _end = time.time()
            history_train.append(np.mean(train_losses))
            print(f"{i_epoch+GLOBAL_EPOCH}/{num_epoch+GLOBAL_EPOCH}: cost {_end-_start:.4f}s train RMSE {np.mean(train_losses):.4f}")
    return (history_train,history_val)

In [19]:
history_train_list = []
history_val_list = []

In [21]:
history_train, history_val = train_function(2)
history_train_list += history_train
history_val_list += history_val
plt.plot(history_train_list[1::5],label="Train")
plt.plot(history_val_list,label="Val")
plt.legend()

RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], use_cuda=True, record_shapes=True) as prof:
    history_train, history_val = train_function(1)
    history_train_list += history_train
    history_val_list += history_val
    plt.plot(history_train_list[1::5],label="Train")
    plt.plot(history_val_list,label="Val")
    plt.legend()
print(prof.key_averages().table(sort_by="cuda_time_total"))

In [ ]:
# print(prof.key_averages().table(sort_by="cpu_time_total"))

In [ ]:
# fig = plt.figure(figsize=(20,20))    
# for i_signal,(signal,y) in enumerate(zip(signals_val,y_val)):
#     snapshot_losses = test_loop(signal, model, loss_f, optimizer, device)
# #                 print(np.mean(val_loss))
# #     val_losses.append(np.mean(snapshot_losses))
#     ts_list = signal.ts_list
#     plt.subplot(len(signals_val)+1,1,i_signal+1)
# #     print(len(snapshot_losses))
#     plt.plot(ts_list,snapshot_losses[:-1])
#     if y[0] == 'dos':
#         plt.axvline(x = float(y[1]), color = 'red', label = y[0])
#     if y[0] == 'privesc':
#         plt.axvline(x = float(y[1]), color = 'blue', label = y[0])
# # plt.show()
# plt.savefig('val_res.png')

In [ ]:
# torch.save(model,f"{GLOBAL_EPOCH}_{np.mean(history_train_list[-1]):.4f}_{np.mean(history_val_list[-1]):.4f}.model")

In [ ]:
# history_train, history_val = train_function(100)
# history_train_list += history_train
# history_val_list += history_val
# plt.plot(history_train_list[1::5],label="Train")
# plt.plot(history_val_list,label="Val")
# plt.legend()

In [ ]:
# states_encoder_val = []
# states_decoder_val = []
# for num_node in nums_node_val:
#     states_encoder_val.append(create_hidden_global(num_node=num_node,out_channels=model.gnn_out_channels))
#     states_decoder_val.append(create_hidden_global(num_node=num_node,out_channels=model.embedding_hidden_nums[-1]))    

# fig = plt.figure(figsize=(20,20))    
# for i_sample, (sample, y, hidden_encoder_global, hidden_decoder_global) in enumerate(zip(X_val, y_val, states_encoder_val, states_decoder_val)):
#     val_loss = test_loop(sample, hidden_encoder_global, hidden_decoder_global, model, loss_f, optimizer, device)
#     # print(np.mean(val_loss))
#     ts_list = [snapshot.timestamp for snapshot in sample]
#     plt.subplot(len(X_val)+1,1,i_sample+1)
#     plt.plot(ts_list,val_loss)
#     if y[0] == 'dos':
#         plt.axvline(x = float(y[1]), color = 'red', label = y[0])
#     if y[0] == 'privesc':
#         plt.axvline(x = float(y[1]), color = 'blue', label = y[0])
# plt.show()

In [ ]:
gru_conv_layers